In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nhl-split-sentences/NHL_cli_notes_split_all_note_types_ka.pkl
/kaggle/input/nhl-reccurence-all-notes-types/nhl_data_clinical_notes_only_all_note_types.pkl
/kaggle/input/nhl-test-pids/NHL_test_patient_ids.pkl


In [2]:
!pip install -q pyspark==3.4.1  spark-nlp==5.3.2 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 5.9 MB/s eta 0:00:000:00:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.0/565.0 kB 32.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-spark-connect 0.5.2 requires google-api-core>=2.19.1, but you have google-api-core 1.34.1 which is incompatible.
google-spark-connect 0.5.2 requires pyspark>=3.5, but you have pyspark 3.4.1 which is incompatible.


In [3]:
import pickle
from datetime import datetime
import sparknlp

from pyspark.ml import PipelineModel
from sparknlp.annotator import *
from sparknlp.base import *

spark = sparknlp.start()

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

:: loading settings :: url = jar:file:/usr/local/lib/python3.11/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-51ddfd51-9f87-4de3-8768-57cff5c0a8b5;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.3.2 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-s3;1.12.500 in central
	found com.amazonaws#aws-java-sdk-kms;1.12.500 in central
	found com.amazonaws#aws-java-sdk-core;1.12.500 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.15 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found software.amazon.ion#ion-java;1.0.2 in central
	found joda-time#joda-time;2.8.1 in central
	found com.amazonaws#jmespath-java;1.12.500 in central
	found com.g

Spark NLP version 5.3.2
Apache Spark version: 3.4.1


In [4]:
import transformers
import torch
from transformers import AutoModelForSequenceClassification

# Config class

In [ ]:
class ml_config:
    sentence_splitter_model_spark = 'sentence_detector_dl'
    patient_cli_notes_data_file_path = './datasources/nhl_data_clinical_notes_only_all_note_types.pkl'
    output_path = ""



# Helper Functions

In [80]:
import os
# Just in case: set env variables from Python as well (but from shell is preferred)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

from pyspark.sql import SparkSession
import sparknlp
from sparknlp.base import DocumentAssembler, LightPipeline
from sparknlp.annotator import SentenceDetectorDLModel
from pyspark.ml import Pipeline

# Create SparkSession with GPU support
spark = sparknlp.start(gpu=True)

documenter = DocumentAssembler().setInputCol("text").setOutputCol("document")
sentencerDL = SentenceDetectorDLModel.pretrained("sentence_detector_dl", "en") \
    .setInputCols(["document"]).setOutputCol("sentences")

sd_pipeline = Pipeline(stages=[documenter, sentencerDL])
model = pipeline.fit(spark.createDataFrame([["test"]]).toDF("text"))
sd_model = LightPipeline(model)


sentence_detector_dl download started this may take some time.


25/07/17 05:13:30 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Approximate size to download 354.6 KB
[OK!]


In [ ]:
documenter = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentencerDL = SentenceDetectorDLModel\
    .pretrained(ml_config.sentence_splitter_model_spark, "en") \
    .setInputCols(["document"]) \
    .setOutputCol("sentences")

sd_pipeline = PipelineModel(stages=[documenter, sentencerDL])

sd_model = LightPipeline(sd_pipeline)

In [82]:
def clinical_notes_sentence_splitter(text):
    return [anno.result for anno in sd_model.fullAnnotate(text)[0]["sentences"]]

# Loading data

In [ ]:
with open(ml_config.patient_cli_notes_data_file_path,'rb') as f:
    nhl_progress_notes_with_label = pickle.load(f)


In [69]:
len(nhl_progress_notes_with_label)

614

In [70]:
nhl_progress_notes_with_label[0][:5] # pat_id, dx_date, start_date, end_date, flt, cli, label

[2,
 datetime.datetime(2007, 6, 25, 0, 0),
 datetime.datetime(2013, 7, 6, 0, 0),
 datetime.datetime(2014, 1, 2, 0, 0),
 '6/30/2008']

In [71]:
# for index,i in enumerate(nhl_progress_notes_with_label):
#     try:
#         print(index,i[0])
#     except:
#         print(i)
#         break
req_data_info[0]

(16, Timestamp('2017-07-11 00:00:00'), Timestamp('2018-01-07 00:00:00'))

In [57]:
dict_recurr = {}
for item in nhl_progress_notes_with_label:
    # try:
        if item:
            if item[6] == 'Y':
                dict_recurr[item[0]] = item[2]


In [72]:
dict_recurr

{5: datetime.datetime(2013, 5, 22, 0, 0),
 28: datetime.datetime(2015, 11, 4, 0, 0),
 33: datetime.datetime(2018, 4, 28, 0, 0),
 44: datetime.datetime(2014, 5, 1, 0, 0),
 46: datetime.datetime(2019, 1, 2, 0, 0),
 79: datetime.datetime(2016, 2, 4, 0, 0),
 89: datetime.datetime(2017, 6, 1, 0, 0),
 125: datetime.datetime(2021, 8, 24, 0, 0),
 138: datetime.datetime(2018, 3, 30, 0, 0),
 202: datetime.datetime(2020, 4, 21, 0, 0),
 245: datetime.datetime(2015, 6, 5, 0, 0),
 257: datetime.datetime(2020, 5, 6, 0, 0),
 282: datetime.datetime(2019, 10, 19, 0, 0),
 299: datetime.datetime(2015, 9, 8, 0, 0),
 303: datetime.datetime(2015, 8, 15, 0, 0),
 319: datetime.datetime(2018, 12, 12, 0, 0),
 332: datetime.datetime(2021, 12, 14, 0, 0),
 335: datetime.datetime(2018, 3, 20, 0, 0),
 337: datetime.datetime(2013, 5, 23, 0, 0),
 363: datetime.datetime(2015, 4, 9, 0, 0),
 372: datetime.datetime(2020, 5, 20, 0, 0)}

In [73]:
nhl_progress_notes_with_label_updated = []
count = 0

for item in nhl_progress_notes_with_label:
    # try:
        if item and (item[4] is None or item[2] >  datetime.strptime(item[4], '%m/%d/%Y')): ## startdate should be greater than flt_date
            pat = item[0]

            if item[4] is None:
                if (item[2] - item[1]).days < 365:
                    count+=1
                    continue
                
    
            if pat in dict_recurr.keys():
                if dict_recurr[pat] < item[2]:
                    count+=1
                    continue
                    
            flt_end_date =  datetime.strptime(item[4], '%m/%d/%Y') if item[4] is not None else item[4]
            start_date = item[2]
            end_date = item[3]
            dx_date = item[1]
    
            if item[6] is None:
                gt = 'N'
            elif item[6] == 'Y':
                gt = 'Y'
            else:
                gt = 'N'
            # end_date = item[2].strftime('%Y-%m-%d')
            nhl_progress_notes_with_label_updated.append((pat, dx_date, start_date, end_date,flt_end_date,item[5], gt))
    # except:
    #     print(item[:4])
    #     break

In [60]:
nhl_progress_notes_with_label_updated[0]

(2,
 datetime.datetime(2007, 6, 25, 0, 0),
 datetime.datetime(2013, 7, 6, 0, 0),
 datetime.datetime(2014, 1, 2, 0, 0),
 datetime.datetime(2008, 6, 30, 0, 0),
 (('CHEST 2 VIEWS (ROUTINE)',
   '*****, ******                 ***: ********    DOB: ********  GENDER: FEMALE TEST:  CHEST 2 VIEWS          DATE:  ******** TIME:  1013 -------------------------------------------------------------------------------- Patient Location:  STH Requesting Physician:  ****** ***** ** -------------------------------------------------------------------------------- Date/Time           Exam Description              ICD-9 Code     PACS Acc # **/**/**** 10:13    CHEST 2 VIEWS                 786.2          ********* -------------------------------------------------------------------------------- EXAMINATION: PA and lateral chest 2 views HISTORY: Persistent cough DATE: **/*/**** COMPARISON: PET CT from */**/**** FINDING: Minimal atelectasis is seen in the lung bases. Otherwise, the lungs are clear. No pleural 

In [74]:
len(nhl_progress_notes_with_label_updated)

542

In [62]:
req_data_info = [
    (item[0], item[1], item[2]#.strftime('%Y-%m-%d')
    )
    for item in req_data_info
]

In [63]:
len(req_data_info)

271

In [64]:
req_data_info[:2]

[(16, Timestamp('2017-07-11 00:00:00'), Timestamp('2018-01-07 00:00:00')),
 (44, Timestamp('2020-02-07 00:00:00'), Timestamp('2020-08-05 00:00:00'))]

In [76]:
labels = []
for sample in nhl_progress_notes_with_label_updated:
    labels.append(sample[6])

np.unique(labels, return_counts =True)

(array(['N', 'Y'], dtype='<U1'), array([522,  20]))

In [24]:
test = []
test_split=[]
for data in tqdm(test_nhl_progress_notes_with_label):
    if data[0]==138 and data[6]=='Y':
        for notes in data[5]:
            test_split.extend(clinical_notes_sentence_splitter(notes[1]))
        test.append([data[0],data[1],data[2], data[3],test_split])   

 28%|██▊       | 66/234 [00:03<00:09, 18.08it/s]


KeyboardInterrupt: 

# Splitting texts into sentences for each patient

In [ ]:
text= 'Subjective:      Patient ID: ***** *** ******* is a 79 y.o. female with history of DLBCL. Chief Complaint:  -- **** is a 79 y.o. lady with recently diagnosed DLBCL, ABC type, double expressor of BCL-2 and BCL-6, stage IIE, with pre-existing low grade B cell lymphoma in **. She is receiving R-miniCHOP. At the time of her DLBCL diagnosis she had a large lung mass as well as cervical LAD. Bx of both her lung and cervical LNs was done and confirmed the diagnosis.  -- Cycle #1 R-miniCHOP was on */**/**.  -- Cycle #2 R-miniCHOP was on */*/**.  -- Cycle #3 R-miniCHOP was on 5*/**/**.  -- Cycle #4 R-miniCHOP on */**/**.  -- PET CT planned on */**/**.  -- Cycle #5 R-**** on */*/**. The patient tolerated the full dose chemo well except for some more fatigue.  -- Restaging PET CT on */**/** showed significant response to treatment, however there was a left renal mass concerning for malignancy (seen also on ***** ** in */****) and an uptake in the thyroid.  -- Cycle #6 R-**** on */**/**.  -- Restaging PET CT on */**/**** showed further improvement in left hilar area as well as the splenic lesion. Because of this, an additional 2 cycles of R-**** were planned, to achieve a total dose of chemotherapy equivalent to 6 cycles of R-****.   -- Cycle #7 R-**** on */**/**.   -- The patient developed heart failure and was hospitalized for volume management. Her LV EF on */**/**** was 45%, down from 63% on */*/**** pre-chemo. Of note, her cumulative dose of doxorubicin after her last dose was 250mg/m2, I.e. within allowable cumulative dose.   -- Adjuvant RT to left hilum in **/****.  -- Restaging PET CT on */*/**** was stable.'


# print(clinical_notes_sentence_splitter(clinical_notes_sentence_splitter(text)))
sent_tokenized = [anno.result for anno in sd_model.fullAnnotate(text)[0]["sentences"]]
# for anno in sd_model.fullAnnotate(text)[0]["sentences"]:

print(sent_tokenized)

In [ ]:
get_topn_reranked_sentences(sent_tokenized,n = 5)

In [ ]:
# pat, dx_date, start_date, end_date,flt_end_date,item[5], gt

In [83]:

test_pat_dates_cli_sentences=[]
for data in tqdm(nhl_progress_notes_with_label_updated):
    split_sentences = []
    for notes in data[5]:
        split_sentences.append(( notes[0], clinical_notes_sentence_splitter(notes[1]))
                              )
    test_pat_dates_cli_sentences.append([data[0],data[1],data[2],data[3], data[4], data[6],split_sentences])

100%|██████████| 542/542 [2:14:24<00:00, 14.88s/it]  


In [ ]:
with open(ml_config.output_path,'wb') as f:
    pickle.dump(test_pat_dates_cli_sentences,f)

In [105]:
len(test_pat_dates_cli_sentences)

542

In [91]:
# test_pat_dates_cli_sentences[1]
import gc

# Delete references to your model and pipeline
del sd_model
del sd_pipeline
del sentencerDL
del documenter

# Run garbage collection to clean up in Python
gc.collect()

# If you're running inside a Spark session and want to be SURE,
# stop the Spark session (this frees all model resources, including on GPU):
spark.stop()